In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error


In [2]:
# Load the datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Show the first few rows
train.head()


,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
0,0,7.817315,576.179270,41.24308670850264,32.135501,4.0,0.803199,37.403527,1.963787,13.691147,62.494044,12.82491203459621,1018.8665053152533,A1,NaN,NaN,0.562096
1,1,24.785727,240.003973,1.3596482765960705,19.977460,8.0,0.479456,21.843315,0.241473,27.545096,43.851238,12.012043660984917,1025.6238537572883,D4,E00,dual-axis,0.396447
2,2,46.652695,687.612799,91.26536837560256,1.496401,4.0,0.822398,48.222882,4.191800,43.363708,NaN,1.814399755560454,1010.9226539809573,C3,E00,NaN,0.573776
3,3,53.339567,735.141179,96.19095521176159,18.491582,3.0,0.837529,46.295748,0.960567,57.720436,67.361473,8.736258932034128,1021.8466633134253,A1,NaN,dual-axis,0.629009
4,4,5.575374,12.241203,27.495073003585226,30.722697,6.0,0.551833,0.000000,0.898062,6.786263,3.632000,0.52268384077164,1008.5559577591927,B2,E00,fixed,0.341874


In [3]:
print("Train Info:")
train.info()
print("\nMissing values in train:\n", train.isnull().sum())

print("\nTest Info:")
test.info()
print("\nMissing values in test:\n", test.isnull().sum())

Train Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  20000 non-null  int64  
 1   temperature         18999 non-null  float64
 2   irradiance          19013 non-null  float64
 3   humidity            20000 non-null  object 
 4   panel_age           18989 non-null  float64
 5   maintenance_count   18973 non-null  float64
 6   soiling_ratio       18990 non-null  float64
 7   voltage             19007 non-null  float64
 8   current             19023 non-null  float64
 9   module_temperature  19022 non-null  float64
 10  cloud_coverage      18990 non-null  float64
 11  wind_speed          20000 non-null  object 
 12  pressure            20000 non-null  object 
 13  string_id           20000 non-null  object 
 14  error_code          14088 non-null  object 
 15  installation_type   14972 non-null  objec

In [4]:
# Convert numeric-looking object columns
cols_to_convert = ['humidity', 'wind_speed', 'pressure']

for col in cols_to_convert:
    train[col] = pd.to_numeric(train[col], errors='coerce')
    test[col] = pd.to_numeric(test[col], errors='coerce')


In [5]:
# Separate features and target
X = train.drop(['efficiency', 'id'], axis=1)
y = train['efficiency']
X_test_final = test.drop('id', axis=1)

# Identify numeric and categorical columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

# Fill numeric features with mean
num_imputer = SimpleImputer(strategy='mean')
X[num_cols] = num_imputer.fit_transform(X[num_cols])
X_test_final[num_cols] = num_imputer.transform(X_test_final[num_cols])

# Fill categorical features with 'Unknown'
cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
X_test_final[cat_cols] = cat_imputer.transform(X_test_final[cat_cols])

In [6]:
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    X_test_final[col] = le.transform(X_test_final[col])

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [9]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [10]:
y_pred = rf.predict(X_val)


In [11]:
mse = mean_squared_error(y_val, y_pred)
score = 100 * (1 - np.sqrt(mse))

print("Validation MSE:", mse)
print("Custom Hackathon Score:", score)

Validation MSE: 0.012000221854765427
Custom Hackathon Score: 89.04544758798177


In [12]:
# 1. Add a new feature: Power = Voltage * Current
X['power'] = X['voltage'] * X['current']
X_test_final['power'] = X_test_final['voltage'] * X_test_final['current']

# 2. Optional: Drop string_id and error_code if they add noise
X = X.drop(['string_id', 'error_code'], axis=1)
X_test_final = X_test_final.drop(['string_id', 'error_code'], axis=1)

# 3. Confirm shape
print(X.shape, X_test_final.shape)


(20000, 14) (12000, 14)


In [15]:
X['temp_diff'] = X['module_temperature'] - X['temperature']
X_test_final['temp_diff'] = X_test_final['module_temperature'] - X_test_final['temperature']


In [16]:
X['irradiance_per_humidity'] = X['irradiance'] / (X['humidity'] + 1)
X_test_final['irradiance_per_humidity'] = X_test_final['irradiance'] / (X_test_final['humidity'] + 1)


In [17]:
X['age_squared'] = X['panel_age'] ** 2
X_test_final['age_squared'] = X_test_final['panel_age'] ** 2


In [18]:
X['wind_pressure_ratio'] = X['wind_speed'] / (X['pressure'] + 1)
X_test_final['wind_pressure_ratio'] = X_test_final['wind_speed'] / (X_test_final['pressure'] + 1)


In [19]:
X['cloud_irradiance'] = X['cloud_coverage'] * X['irradiance']
X_test_final['cloud_irradiance'] = X_test_final['cloud_coverage'] * X_test_final['irradiance']


In [ ]:
#using random forest regressor with feature engineering

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Add new features to training and test data
X['temp_diff'] = X['module_temperature'] - X['temperature']
X_test_final['temp_diff'] = X_test_final['module_temperature'] - X_test_final['temperature']

X['irradiance_per_humidity'] = X['irradiance'] / (X['humidity'] + 1)
X_test_final['irradiance_per_humidity'] = X_test_final['irradiance'] / (X_test_final['humidity'] + 1)

X['age_squared'] = X['panel_age'] ** 2
X_test_final['age_squared'] = X_test_final['panel_age'] ** 2

X['wind_pressure_ratio'] = X['wind_speed'] / (X['pressure'] + 1)
X_test_final['wind_pressure_ratio'] = X_test_final['wind_speed'] / (X_test_final['pressure'] + 1)

X['cloud_irradiance'] = X['cloud_coverage'] * X['irradiance']
X_test_final['cloud_irradiance'] = X_test_final['cloud_coverage'] * X_test_final['irradiance']

# Split train/val set
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
model = RandomForestRegressor(n_estimators=150, max_depth=10, random_state=42)
model.fit(X_train_split, y_train_split)

# Predict and Evaluate
val_preds = model.predict(X_val_split)
mse = mean_squared_error(y_val_split, val_preds)
score = 100 * (1 - np.sqrt(mse))

print(f"Validation MSE: {mse}")
print(f"Custom Hackathon Score: {score}")


Validation MSE: 0.011670978244882512
Custom Hackathon Score: 89.19676981413312
